In [9]:
import os
import json
import ftplib

with open("credentials/passwords.json", "r") as file:
    passwords = json.load(file)

class InvalidPathError(Exception):
    pass

class Bluehost:
    # with Bluehost() as bh:
    #     local_path = os.path.join(".", "index.html")
    #     remote_path = os.path.join(".", "index.html")
    #     print(bh.listdir())
    #     bh.upload_file(local_path, remote_path, overwrite=True)
    #     bh.download_file(remote_path, local_path, overwrite = True)

    def __init__(self):
        self.ftp = None

    def __enter__(self):
        self.ftp = self.login()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        try:
            if self.ftp:
                self.ftp.quit()
        finally:
            self.ftp = None
        return False

    def login(self):
        ftp_server = 'ftp.luh.sgg.mybluehost.me'
        ftp_username = 'mpuche2@luh.sgg.mybluehost.me'
        ftp_password = passwords["BLUEHOST"]
        self.ftp = ftplib.FTP(ftp_server)
        try:
            self.ftp.login(ftp_username, ftp_password)
            welcome_message = self.ftp.getwelcome()
            if welcome_message:
                print("===")
                print("FTP connection established successfully!")
                print("Welcome message:", welcome_message)
                print("===")
            else:
                print("FTP connection failed. Check your credentials.")
        except ftplib.all_errors as e:
            print("FTP connection failed:", e)
            self.ftp.quit()
        finally:
            return self.ftp

    def logout(self):
        if hasattr(self, 'ftp'):
            self.ftp.quit() 

    def file_exists(self, path):
        try:
            self.ftp.size(path)
            return True
        except ftplib.error_perm:
            return False

    def dir_exists(self, path):
        try:
            self.ftp.nlst(path)
            return True
        except ftplib.error_perm:
            return False

    def listdir_recursive_only_files(self, path = os.path.join("")):
        try:
            listdir = self.ftp.nlst(path)
        except Exception as e:
            print(f"Error listing dir {path}: {e}")
            return []

        cleaned_listdir = []
        for x in listdir:
            if x[0] == "." or x[-1] == ".":
                continue
            elif x.count('.') > 0:
                cleaned_listdir.append(x)
            else:
                cleaned_listdir.extend(self.listdir_recursive_only_files(x))

        cleaned_listdir = sorted(cleaned_listdir)
        return cleaned_listdir

    def listdir(self, path = os.path.join("")):
        try:
            listdir = self.ftp.nlst(path)
        except Exception as e:
            print(f"Error listing dir {path}: {e}")
            return []

        cleaned_listdir = []
        for x in listdir:
            if x == "." or x[-1] == "." or x == "cgi-bin":
                continue
            cleaned_listdir.append(x)

        cleaned_listdir = sorted(cleaned_listdir)
        return cleaned_listdir

    def upload_file(self, local_path, remote_path, overwrite=False):
        local_dir = os.path.dirname(local_path)
        remote_dir = os.path.dirname(remote_path)
        local_dir = local_dir if local_dir != "" else os.path.join(".")
        remote_dir = remote_dir if local_dir != "" else os.path.join(".")
        if not os.path.exists(local_dir):
            raise InvalidPathError(f"Local path dir doesn't exist: {local_dir}")
        if not os.path.exists(local_path):
            raise InvalidPathError(f"Local path doesn't exist: {local_path}")
        if not self.dir_exists(remote_dir):
            raise InvalidPathError(f"Remote dir doesn't exist: {remote_dir}")
        if not overwrite and self.file_exists(remote_path):
            raise InvalidPathError(f"Remote file already exists: {remote_path}")
        with open(local_path, 'rb') as file:
            response = self.ftp.storbinary(f"STOR {remote_path}", file)
        print("---")
        print(f"filename: {os.path.basename(local_path)}")
        print(f"local path: {local_path}")
        print(f"remote path: {remote_path}")
        print(f"response: {response}")

    def download_file(self, remote_path, local_path, overwrite=False):
        local_dir = os.path.dirname(local_path)
        remote_dir = os.path.dirname(remote_path)
        local_dir = local_dir if local_dir != "" else os.path.join(".")
        remote_dir = remote_dir if local_dir != "" else os.path.join(".")
        if not os.path.exists(local_dir):
            raise InvalidPathError(f"Local dir doesn't exist: {local_dir}")
        if not overwrite and os.path.exists(local_path):
            raise InvalidPathError(f"Local file already exist: {local_path}")
        if not self.dir_exists(remote_dir):
            raise InvalidPathError(f"Remote dir doesn't exist: {remote_dir}")
        if not self.file_exists(remote_path):
            raise InvalidPathError(f"Remote file doesn't exists: {remote_path}")
        with open(local_path, 'wb') as file:
            response = self.ftp.retrbinary('RETR ' + remote_path, file.write)
        print("---")
        print(f"filename: {os.path.basename(local_path)}")
        print(f"local path: {local_path}")
        print(f"remote path: {remote_path}")
        print(f"response: {response}")

    def transfer_mp3_files(self, book):
        local_dir = os.path.join(".", 'drive', "MyDrive", "BOOKS", book, "AUDIO", "AUDIO_SENTENCES")
        remote_dir = os.path.join(".", "audio", "books", book)
        if not os.path.exists(local_dir):
            raise InvalidPathError(f"Local dir doesn't exist: {local_dir}")
        if not self.dir_exists(remote_dir):
            raise InvalidPathError(f"Remote dir doesn't exist: {remote_dir}")

        local_dir_filenames = []
        local_filenames = sorted(os.listdir(local_dir))
        for local_filename in local_filenames:
            pattern = r'^B(\d{3})C(\d{3})S(\d{3})_echo\.mp3$'
            if re.match(pattern, local_filename):
                local_dir_filenames.append(local_filename)

        remote_dir_filenames = []
        remote_paths = sorted(self.listdir(remote_dir))
        for remote_path in remote_paths:
            remote_filename = os.path.basename(remote_path)
            pattern = r'^B(\d{3})C(\d{3})S(\d{3})_echo\.mp3$'
            if re.match(pattern, remote_filename):
                remote_dir_filenames.append(remote_filename)

        pending_filenames = sorted(list(set(local_dir_filenames) - set(remote_dir_filenames)))
        for filename in pending_filenames:
            local_path = os.path.join(local_dir, filename)
            remote_path = os.path.join(remote_dir, filename)
            if not os.path.exists(local_path):
                raise InvalidPathError(f"Local file doesn't exist: {local_path}")
            if not self.file_exists(remote_path):
                with open(local_path, 'rb') as file:
                    response = self.ftp.storbinary(f"STOR {remote_path}", file)
                print("---")
                print(f"filename: {filename}")
                print(f"local path: {local_path}")
                print(f"remote path: {remote_path}")
                print(f"response: {response}")

In [10]:
def download_transcripts_from_bluehost():
    with Bluehost() as bh:
        # contents = bh.listdir(os.path.join(".", "transcriptions", "books", book))
        # print(contents)
        for book in ["B001", "B002", "B009"]:
            path = os.path.join(".", "transcriptions", "books", book, f"{book}_TRANS_ALL.txt")
            bh.download_file(
                remote_path = path,
                local_path = path,
                overwrite=True
            )

def update_website_in_bluehost():
    with Bluehost() as bh:
        for filename in ["index.html", "style.css", "script_slow.js", "script_fast.js"]:
            path = os.path.join(".", filename)
            bh.upload_file(
                remote_path = path,
                local_path = path,
                overwrite=True
            )

def create_folder():
    bh = Bluehost()
    ftp = bh.login()
    ftp.mkd('filters')
    ftp.close()

def upload_file():
    path = os.path.join(".", "filters", "filters.txt")
    with Bluehost() as bh:
        bh.upload_file(
            local_path = path,
            remote_path = path,
        )

def deleteme01():
    text = ""
    for book in ["B001", "B002", "B009"]:
        path = os.path.join(".", "transcriptions", "books", book, f"{book}_TRANS_ALL.txt")
        assert os.path.exists(path)
        with open(path, "r") as file:
            text += file.read()

    ans = ""
    lines = text.split("\n")
    for line in lines:
        if line.startswith("B0"):
            ans += "[o] " + line + "\n"

    with open("deleteme.txt", "w") as file:
        file.write(ans)

def deleteme02():
    with open("./filters/filters.txt", "r") as file:
        text = file.read(ans)

    lines = text.split("\n")
    for line in lines:
        if line.startswith("[0]"):
            ans += line + "\n"

    with open("./filters/included.txt", "w") as file:
        text = file.write(ans)

In [17]:
# update_website_in_bluehost()

In [12]:
# path = os.path.join(".", "filters", "filters.txt")
# with Bluehost() as bh:
#     bh.upload_file(
#         remote_path = path,
#         local_path = path,
#         # overwrite=True
#     )

In [13]:
# bh = Bluehost()
# ftp = bh.login()

In [14]:
# path = os.path.join(".", "text", "books")
# bh.listdir(path)

In [15]:
# book = "B009"
# path = os.path.join(".", "text", "books", book, f"{book}_TEXTS_ALL.txt")
# bh.upload_file(
#     local_path = path,
#     remote_path = path
# )

In [16]:
# bh.logout()